In [33]:
%%time

import os
import pandas as pd
import xml.etree.ElementTree as ET


def remove_ns_prefix(tree):
    root = tree.getroot()

    # Récupérer l'espace de noms TEI
    nsmap = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # Supprimer les préfixes mais conserver les espaces de noms
    for elem in root.iter():
        if '}' in elem.tag:
            elem.tag = elem.tag.split('}', 1)[1]  # Supprime le préfixe

    # Ajouter l'espace de noms TEI en tant que namespace par défaut
    ET.register_namespace('', nsmap['tei'])  # Définit "http://www.tei-c.org/ns/1.0" comme défaut

    return tree


# Chemins des fichiers
dossier_xml = "../"
fichier_csv = "../archive/Inventaire_originaux.csv"

# Chargement du fichier CSV
df = pd.read_csv(fichier_csv, sep=';', dtype=str)  # Assurer la lecture en texte

# Parcours des fichiers XML
for fichier in os.listdir(dossier_xml):
    print(fichier)
    if fichier.endswith(".xml"):
        chemin_fichier = os.path.join(dossier_xml, fichier)
        
        # Chargement du fichier XML
        tree = ET.parse(chemin_fichier)
        root = tree.getroot()
        
        # Espace de noms TEI (si nécessaire)
        ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
        
        # Chercher l'élément witness[@n='A']
        witness = root.find(".//tei:sourceDesc/tei:listWit/tei:witness[@n='A']/tei:msDesc/tei:msIdentifier/tei:idno", ns)
        
        if witness is not None and witness.text:
            texte_witness = witness.text.strip()
            
            # Vérifier la correspondance avec le fichier CSV
            for _, row in df.iterrows():
                
                attendu = f"{row['Cote']}, pièce {row['N° de pièce']}"
                
                if texte_witness == attendu and row['Auteur']:
                    # Chercher profileDesc et ajouter textClass
                    profile_desc = root.find(".//tei:profileDesc", ns)
                    
                    if profile_desc is not None:
                        text_class = ET.Element("textClass")
                        text_class.text = str(row['Auteur'])
                        profile_desc.append(text_class)

                        # Appliquer la fonction pour nettoyer l'affichage de l'espace de nom
                        tree = remove_ns_prefix(tree)
                        
                        
                        # Sauvegarde du fichier modifié
                        tree.write(chemin_fichier, encoding="utf-8", xml_declaration=True)
                        print(f"Ajout de <textClass> dans {fichier}")

                    break


archive
biblio.xml
css
README.md
sauvegarde
x1117_d1e209543.xml
x1131_d1e209711.xml
x1136_d1e209924.xml
x1136_d1e210024.xml
x1136_d1e210405.xml
x1139_d1e1415.xml
x1139_d1e210214.xml
x1142_d1e212901.xml
x1142_d1e213044.xml
x1142_d1e213907.xml
x1142_d1e3004.xml
Ajout de <textClass> dans x1142_d1e3004.xml
x1143_d1e215333.xml
x1144_d1e214001.xml
x1145_d1e214444.xml
x1146_d1e214796.xml
x1146_d1e214934.xml
x1147_d1e215004.xml
x1148_d1e215192.xml
x1148_d1e221979.xml
x1148_d1e6048.xml
Ajout de <textClass> dans x1148_d1e6048.xml
x1148_d1e8175.xml
Ajout de <textClass> dans x1148_d1e8175.xml
x1149_d1e11074.xml
Ajout de <textClass> dans x1149_d1e11074.xml
x1149_d1e222237.xml
x1151_d1e14326.xml
Ajout de <textClass> dans x1151_d1e14326.xml
x1152_d1e16319.xml
Ajout de <textClass> dans x1152_d1e16319.xml
x1154_d1e222421.xml
x1154_d1e222627.xml
x1157_d1e222774.xml
x1159_d1e19041.xml
Ajout de <textClass> dans x1159_d1e19041.xml
x1162_d1e223042.xml
x1162_d1e223366.xml
x1162_d1e223532.xml
x1162_d1e223815.

In [ ]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

# Suppression des préfixes tout en gardant le namespace
def remove_ns_prefix(tree):
    root = tree.getroot()
    nsmap = {'tei': 'http://www.tei-c.org/ns/1.0'}
    for elem in root.iter():
        if '}' in elem.tag:
            elem.tag = elem.tag.split('}', 1)[1]
    ET.register_namespace('', nsmap['tei'])
    return tree

# Chargement du fichier CSV
df = pd.read_csv("../archive/Inventaire_originaux.csv", sep=';', dtype=str)

# Dictionnaires pour taxonomies
actions_dict = {k: f"#{k.replace(' ', '_').replace('-', '_').lower()}" for k in df["Action juridique"].dropna().unique()}
titres_dict = {k: f"#{k.replace(' ', '_').replace('-', '_').lower()}" for k in df["Auteur"].dropna().unique()}

# Parcours des fichiers XML
dossier_xml = "../"
for fichier in os.listdir(dossier_xml):
    if fichier.endswith(".xml"):
        chemin_fichier = os.path.join(dossier_xml, fichier)
        
        tree = ET.parse(chemin_fichier)
        root = tree.getroot()
        
        ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
        witness = root.find(".//tei:sourceDesc/tei:listWit/tei:witness[@n='A']/tei:msDesc/tei:msIdentifier/tei:idno", ns)
        
        if witness is not None and witness.text:
            texte_witness = witness.text.strip()
            for _, row in df.iterrows():
                attendu = f"{row['Cote']}, pièce {row['N° de pièce']}"
                
                if texte_witness == attendu:
                    profile_desc = root.find(".//tei:profileDesc", ns)
                    if profile_desc is not None:
                        text_class = ET.Element("textClass")
                        
                        if row["Auteur"] in titres_dict:
                            ET.SubElement(text_class, "catRef", target=titres_dict[row["Auteur"]])
                        if row["Action juridique"] in actions_dict:
                            ET.SubElement(text_class, "catRef", target=actions_dict[row["Action juridique"]])

                        profile_desc.append(text_class)
                        tree = remove_ns_prefix(tree)
                        tree.write(chemin_fichier, encoding="utf-8", xml_declaration=True)
                        print(f"Ajout de <textClass> dans {fichier}")
